<a href="https://colab.research.google.com/github/mhoangvslev/DECA/blob/master/Detailed_Expression_Capture_and_Animation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
#@title Setup dependencies

%cd /content/
!git clone https://github.com/mhoangvslev/DECA

%cd DECA/
!apt -q install -y zip unzip ffmpeg libsm6 libxext6
#!pip install -r requirements.txt
#!pip install 'torch==1.6.0'
#!pip install 'torchvision==0.7.0'
#!pip install -q 'pytorch3d==0.2.5'
!pip install -q numpy scipy chumpy scikit-image opencv-python PyYAML face-alignment yacs kornia ninja fvcore gdown matplotlib
!pip install -q gdown==4.5.4 --no-cache-dir
#!pip install --upgrade ipykernel

In [ ]:
#@title Install pytorch3d for Colab
# SOURCE: https://github.com/facebookresearch/pytorch3d/blob/main/INSTALL.md
import sys
import torch
pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])
!pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html


In [ ]:
#@title Download models
#@markdown By executing this cell, you agree to the [LICENSE](https://flame.is.tue.mpg.de/modellicense.html) provided by Max-Planck-Gesellschaft zur Förderung der Wissenschaften e.V

print("Downloading FLAME2020 model...")
!gdown 1FRoGBmNCLM6Q0FyP_IeqRtbwTsHslss5 -O FLAME2020.zip
!unzip -o FLAME2020.zip -d data/

print("Downloading deca_model...")
!gdown 1rp8kdyLPvErw2dTmqtjISRVvQLj6Yzje -O data/deca_model.tar

# Face reconstruction

In [ ]:
!python -V

In [ ]:
!pip list | grep torch

In [ ]:
#@title Run paper demo
print("Setting up...")
!pip list | grep torch
#!pip install -q kornia==0.4.0 yacs==0.1.8 face_alignment ninja fvcore

print("Check for NVIDIA Driver...")
!nvidia-smi

print("Running experiments...")
import os
input_folder = "TestSamples/AFLW2000" #@param {type:"string"}
output_folder = os.path.join(input_folder, "results")
!python demos/demo_reconstruct.py -i $input_folder -s $output_folder --saveDepth True --saveObj True


In [ ]:
#@title Use your own image
#@markdown Upload your images to `upload` folder under `DECA`

print("Check for NVIDIA Driver...")
!nvidia-smi

print("Running experiments...")
import os
input_folder = "/content/" #@param {type:"string"}
output_folder = os.path.join(input_folder, "results")
!python demos/demo_reconstruct.py -i $input_folder -s $output_folder --saveDepth True --saveObj True


In [ ]:
#@title Visualize the results
#@markdown Add import ctypes.util in glcontext.py if error. Warning: May crash your session!
#@markdown Alternatively, you can use this [3dviewer.net](https://3dviewer.net/).
import matplotlib.pyplot as plt

# Util function for loading meshes
from pytorch3d.io import load_objs_as_meshes, load_obj

# Data structures and functions for rendering
from pytorch3d.structures import Meshes
from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene
from pytorch3d.vis.texture_vis import texturesuv_image_matplotlib
from pytorch3d.renderer import (
    look_at_view_transform,
    FoVPerspectiveCameras, 
    PointLights, 
    DirectionalLights, 
    Materials, 
    RasterizationSettings, 
    MeshRenderer, 
    MeshRasterizer,  
    SoftPhongShader,
    TexturesUV,
    TexturesVertex
)
import os

import tensorflow as tf

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")

# Initialize a camera.
# With world coordinates +Y up, +X left and +Z in, the front of the cow is facing the -Z direction. 
# So we move the camera by 180 in the azimuth direction so it is facing the front of the cow. 
R, T = look_at_view_transform(2.7, 0, 180) 
cameras = FoVPerspectiveCameras(device=device, R=R, T=T)

# Define the settings for rasterization and shading. Here we set the output image to be of size
# 512x512. As we are rendering images for visualization purposes only we will set faces_per_pixel=1
# and blur_radius=0.0. We also set bin_size and max_faces_per_bin to None which ensure that 
# the faster coarse-to-fine rasterization method is used. Refer to rasterize_meshes.py for 
# explanations of these parameters. Refer to docs/notes/renderer.md for an explanation of 
# the difference between naive and coarse-to-fine rasterization. 
raster_settings = RasterizationSettings(
    image_size=512, 
    blur_radius=0.0, 
    faces_per_pixel=1, 
)

# Place a point light in front of the object. As mentioned above, the front of the cow is facing the 
# -z direction. 
lights = PointLights(device=device, location=[[0.0, 0.0, -3.0]])

# Create a Phong renderer by composing a rasterizer and a shader. The textured Phong shader will 
# interpolate the texture uv coordinates for each vertex, sample from a texture image and 
# apply the Phong lighting model
renderer = MeshRenderer(
    rasterizer=MeshRasterizer(
        cameras=cameras, 
        raster_settings=raster_settings
    ),
    shader=SoftPhongShader(
        device=device, 
        cameras=cameras,
        lights=lights
    )
)

results = next(os.walk(output_folder))[1]
for folder in results:
  filename = os.path.join(output_folder, folder)
  mesh = load_objs_as_meshes([os.path.join(filename, f'{folder}.obj')], device=device)
  #clr_map = load(os.path.join(filename, f'{folder}.png'))
  images = renderer(mesh)
  plt.figure(figsize=(10, 10))
  plt.imshow(images[0, ..., :3].cpu().numpy())
  plt.axis("off");
  break


In [ ]:
#@title Download the result
import os
from google.colab import files

print(next(os.walk(output_folder)))
folders = [ os.path.join(output_folder, f) for f in next(os.walk(os.path.join(input_folder, 'results')))[1] ]

print(f'Download results...')
os.system(f'zip -r download.zip {" ".join(folders)}')
files.download("download.zip")